A nice way to understand how CNNs work is to build a simple model from scratch, which can be done using just Numpy. Here, we will go through the process and train the model on MNIST, structuring the model similarly to the original LeNet.

Note that this is meant to serve as a guide; just copying code will not get you anywhere. First, based on the concepts of CNN that you have learnt, try to complete the code with the hints given. The full code is there to guide you only if you get stuck.

Every Convolutional Neural Network has a few fundamental layers -  convolutional layers, max-pooling layers, ReLU activation, and a dense neural net. Here, we start coding these layers, step by step. First, we write the forward pass for the convolutional layer

In [ ]:
import numpy as np

class Conv:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=1):
        self.stride = stride
        self.padding = padding
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        # In this implementation we have assumed a square filter with both dimensions equalling kernel_size
        self.weights = np.random.randn(out_channels, in_channels, kernel_size, kernel_size)*0.001
        self.bias = np.zeros(out_channels)

    def forward(self, x):
        # First, we pad the input to maintain consistency in the input and output dimensions after applying the convolutional filter
        pad = self.padding
        stride = self.stride
        k = self.kernel_size
        weights = self.weights
        # Assuming input x has shape (batch_size, in_channels, height, width)
        batch_size, in_channels, dimx, dimy = x.shape
        x = np.pad(x, ((0, 0), (0, 0), (pad, pad), (pad, pad)), mode='constant')
        self.x = x
        dimx_padded = x.shape[2]
        dimy_padded = x.shape[3]
        # We initialize the output matrix where we store the product values
        out_channels = self.out_channels
        out_height = (dimx_padded - k) // stride + 1
        out_width = (dimy_padded - k) // stride + 1
        output = np.zeros((batch_size, out_channels, out_height, out_width))

        # We make slices of x, for which we use step size = stride and slice width as kernel size
        # Think about the range for i and j while slicing, it is crucial to this step
        # YOUR CODE GOES HERE to loop through batches, output channels, and spatial dimensions (height and width)
            # YOUR CODE GOES HERE to extract the patch from the padded input
            # To apply a convolution, we multiply element-wise each value of the filter matrix with the slice, and also add the bias afterwards
            # YOUR CODE GOES HERE to compute the convolution for the current patch and weight/bias
            # YOUR CODE GOES HERE to store the result in the output matrix
        return output

In [ ]:
import numpy as np

class Conv:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=1):
        self.stride = stride
        self.padding = padding
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        # In this implementation we have assumed a square filter with both dimensions equalling kernel_size
        self.weights = np.random.randn(out_channels, in_channels, kernel_size, kernel_size)*0.001
        self.bias = np.zeros(out_channels)

    def forward(self, x):
        # First, we pad the input to maintain consistency in the input and output dimensions after applying the convolutional filter
        pad = self.padding
        stride = self.stride
        k = self.kernel_size
        weights = self.weights
        # Assuming input x has shape (batch_size, in_channels, height, width)
        batch_size, in_channels, dimx, dimy = x.shape
        x = np.pad(x, ((0, 0), (0, 0), (pad, pad), (pad, pad)), mode='constant')
        self.x = x
        dimx_padded = x.shape[2]
        dimy_padded = x.shape[3]
        # We initialize the output matrix where we store the product values
        out_channels = self.out_channels
        out_height = (dimx_padded - k) // stride + 1
        out_width = (dimy_padded - k) // stride + 1
        output = np.zeros((batch_size, out_channels, out_height, out_width))

        # We make slices of x, for which we use step size = stride and slice width as kernel size
        # Think about the range for i and j while slicing, it is crucial to this step
        for b in range(batch_size):
            for c_out in range(out_channels):
                for i in range (0, dimx_padded-k+1, stride):
                    for j in range(0, dimy_padded-k+1, stride):
                        patch = x[b, :, i:i+k, j:j+k]
                        # To apply a convolution, we multiply element-wise each value of the filter matrix with the slice, and also add the bias afterwards
                        prod = weights[c_out, :, :, :] * patch
                        val = np.sum(prod) + self.bias[c_out]
                        output[b, c_out, i//stride, j//stride] = val
        return output

Now, we will move on to writing the forward pass for a max-pooling layer. This layer reduces the dimensionality of the output relative to the input by retaining information about the maximum values in each chunk. In a sense, it retains the most relevant information, balancing a tradeoff between information loss and the time required for processing.
Note that we wish to store information about the indices of the maximum value in each chunk, as this will come to be useful during the backpropagation.

In [ ]:
import numpy as np
# Here, we maxpool assuming a 3D array x
class MaxPool:
    def __init__(self, kernel_size=2, stride=2):
        self.kernel_size = kernel_size
        self.stride = stride

    def forward(self, x):
        kernel_size = self.kernel_size
        stride = self.stride
        batch_size, channels, height, width = x.shape
        output_height = (height - kernel_size) // stride + 1
        output_width = (width - kernel_size) // stride + 1
        output = np.zeros((batch_size, channels, output_height, output_width))
        self.max_mask = np.zeros_like(x)
        self.max_indices = np.zeros((batch_size, channels, output_height, output_width, 2), dtype=int)

        # YOUR CODE GOES HERE to loop through batches, channels, and spatial dimensions (height and width)
            # YOUR CODE GOES HERE to extract the chunk from the input
            # YOUR CODE GOES HERE to compute the maximum value in the chunk
            # YOUR CODE GOES HERE to store the maximum value in the output matrix
            # YOUR CODE GOES HERE to find the indices of the maximum value within the chunk
            # YOUR CODE GOES HERE to store the mask and indices of the maximum value
        return output

In [ ]:
import numpy as np
# Here, we maxpool assuming a 3D array x
class MaxPool:
    def __init__(self, kernel_size=2, stride=2):
        self.kernel_size = kernel_size
        self.stride = stride

    def forward(self, x):
        kernel_size = self.kernel_size
        stride = self.stride
        batch_size, channels, height, width = x.shape
        output_height = (height - kernel_size) // stride + 1
        output_width = (width - kernel_size) // stride + 1
        output = np.zeros((batch_size, channels, output_height, output_width))
        self.max_mask = np.zeros_like(x)
        self.max_indices = np.zeros((batch_size, channels, output_height, output_width, 2), dtype=int)

        for b in range(batch_size):
            for c in range(channels):
                for i in range(0, height - kernel_size + 1, stride):
                    for j in range(0, width - kernel_size + 1, stride):
                        chunk = x[b, c, i:i + kernel_size, j:j + kernel_size]
                        output[b, c, i // stride, j // stride] = np.max(chunk)
                        flat_index = np.argmax(chunk)
                        i_max, j_max = np.unravel_index(flat_index, chunk.shape)
                        self.max_mask[b, c, i + i_max, j + j_max] = 1
                        self.max_indices[b, c, i // stride, j // stride] = (i + i_max, j + j_max)
        return output

Now, we quickly define the ReLU class (as you shall see, defining it as a class is useful as it allows us to implement common forward pass and back-propagation methods through all the layers.

In [ ]:
class ReLU:
    def __init__(self):
        self.x = None

    def forward(self, x):
    # Here, x is a numpy array of arbitrary dimension
        # YOUR CODE GOES HERE to apply the ReLU activation function
        pass

In [ ]:
class ReLU:
    def __init__(self):
        self.x = None

    def forward(self, x):
        self.x = x
    # Here, x is a numpy array of arbitrary dimension
        return np.maximum(0, x)

Before we pass data to the dense neural network layer, it is flattened into a 1D array (or in the case of batching, a 2D array with a 1D array associated with each image in the batch). We now define a class that performs this function.

In [ ]:
class Flatten:
    def __init__(self):
        self.original_shape = None

    def forward(self, x):
        self.original_shape = x.shape
        # The reshape should flatten the dimensions after the batch size
        return x.reshape(x.shape[0], -1)

Now, the flattened data is to be passed through a dense neural net layer. We define the class associated in the code below (the logic is the same as was covered in prior weeks).
Please note, that while training the model, an issue was discovered that initializing weights as zeros or using the NumPy random function led to repeated warnings about exploding gradients. Hence, we took inspiration from the original LeNet (1998) paper and decided to initialize weights by selecting from a distribution as defined below. You are encouraged to try different initializations to explore the issues that arise.

In [ ]:
import numpy as np

class DenseNNLayer:
    def __init__(self, input_features, output_features):
        self.in_features = input_features
        self.out_features = output_features

        # Initializing weights as zeros caused gradients to explode
        # self.weights = np.zeros((self.out_features, self.in_features+1))
        # self.weights = np.random.randn(self.out_features, self.in_features + 1) * 0.0001
        limit = 1 / np.sqrt(self.in_features)
        self.weights = np.random.uniform(-limit, limit, (self.out_features, self.in_features + 1))

    def forward(self, x):
        weights = self.weights
        B = x.shape[0]
        # YOUR CODE GOES HERE to add a bias term to the input (augmenting the input with a column of ones)
        x_augmented = None # Replace with your code
        self.x_augmented = x_augmented
        # YOUR CODE GOES HERE to perform the matrix multiplication for the forward pass, considering the batch dimension
        output = None # Replace with your code
        return output

In [ ]:
class DenseNNLayer:
    def __init__(self, input_features, output_features):
        self.in_features = input_features
        self.out_features = output_features

        # Initializing weights as zeros caused gradients to explode
        # self.weights = np.zeros((self.out_features, self.in_features+1))
        # self.weights = np.random.randn(self.out_features, self.in_features + 1) * 0.0001
        limit = 1 / np.sqrt(self.in_features)
        self.weights = np.random.uniform(-limit, limit, (self.out_features, self.in_features + 1))
    def forward(self, x):
        weights = self.weights
        B = x.shape[0]
        x_augmented = np.concatenate((np.ones((B, 1)), x), axis=1)
        self.x_augmented = x_augmented
        output = x_augmented @ weights.T #Here, x_augmented has dimension batchsize * input_features+1, weights.T has dimension input_features+1, output_features
        return output

Now, we need a loss function. Here, we use cross-entropy, which is common in classification problems. We need to first normalize the activations by passing them through a softmax layer.

In [ ]:
class SoftMax:
    def __init__(self):
        pass

    def forward(self, x):
      # x has dimensions (batch size, output features)
        # YOUR CODE GOES HERE to apply the softmax function
        pass

In [ ]:
class SoftMax:
    def __init__(self):
        pass

    def forward(self, x):
      # x has dimensions (batch size, output features)
        x_shifted = x - np.max(x, axis=1, keepdims=True) # Doesn't change the result, just results in smaller exponentials preventing potential overflow issues
        exp_x = np.exp(x_shifted)
        probabilities = exp_x / np.sum(exp_x, axis=1, keepdims=True)
        return probabilities

As a side note, its important to understand broadcasting rules for NumPy here.
https://medium.com/@weidagang/understanding-broadcasting-in-numpy-c44dceae42ea
This is a great starting point to get clarity. Here, we had to use keepdims=True to get an output from np.max of dimension (batch_size, 1); otherwise, we would have a 1D array of length batch_size, which can not be broadcasted to x. A similar logic applies to probabilities.

Here, we compute the cross-entropy loss for the batch, and take the mean as the loss function we are trying to minimize. The following link provides a refresher if needed.
https://www.geeksforgeeks.org/machine-learning/what-is-cross-entropy-loss-function/

In [ ]:
import numpy as np
# Here, we calculate the cross entropy loss for the output from the dense layer after applying softmax
class CrossEntropy:
    def __init__(self):
        pass

    def forward(self, x, y):
        self.x = x
        self.y = y
        #x is the array of predicted probabilities, while y consists of true labels
        # We calculate the cross entropy loss for each sample in the batch and then take the mean
        # YOUR CODE GOES HERE to calculate the cross entropy loss for each sample in the batch
        cross_entropy = None  # Replace with your code
        # YOUR CODE GOES HERE to take the mean across the batch
        loss = None # Replace with your code
        return loss

In [ ]:
class CrossEntropy:
    def __init__(self):
        pass

    def forward(self, x, y):
        self.x = x  # store for backward
        self.y = y

        # Small epsilon to prevent log(0)
        eps = 1e-15
        clipped_x = np.clip(x, eps, 1 - eps)  # optional but safer than log(x + eps)

        # Compute per-sample loss: shape (B,)
        loss_per_sample = -np.sum(y * np.log(clipped_x), axis=1)

        # Mean over batch
        loss = np.mean(loss_per_sample)

        return loss

This is the first major milestone completed for a basic CNN - we have completed the forward pass for all the major layers that we needed. Now, we work on the backpropagation of gradients, which will be used for batch gradient descent.

The first gradient we compute is the gradient of the cross-entropy loss with respect to the output values (logits) provided to the softmax function. Note that we are combining two steps here - the gradient w.r.t. the cross-entropy loss function inputs, and the gradient of these inputs w.r.t the softmax input logits. This is because it greatly simplifies the mathematics involved, and we do not need the intervening gradients as there are no parameters to be tuned in this layer. This is explained in the derivation below.

### Gradient of Cross-Entropy Loss with Softmax

Let:
- $z \in \mathbb{R}^C$: input logits (output of the neural net)
- $y \in \mathbb{R}^C$: one-hot true label
- $s = \text{softmax}(z) \in \mathbb{R}^C$, where:

$$
s_i = \frac{e^{z_i}}{\sum_k e^{z_k}}
$$

---

### Cross-Entropy Loss:

$$
\mathcal{L} = -\sum_i y_i \log(s_i)
$$

Substitute $s_i$:

$$
\mathcal{L} = -\sum_i y_i \log\left( \frac{e^{z_i}}{\sum_k e^{z_k}} \right)
= -\sum_i y_i (z_i - \log \sum_k e^{z_k})
= -\sum_i y_i z_i + \log \sum_k e^{z_k}
$$
##### Note: $$
\sum_i y_i \log \sum_k e^{z_k} = (\sum_i y_i) \log \sum_k e^{z_k} = \log \sum_k e^{z_k}
$$
---

### Now Take Derivative w.r.t. $z_j$:

$$
\frac{\partial \mathcal{L}}{\partial z_j}
= -y_j + \frac{e^{z_j}}{\sum_k e^{z_k}} = s_j - y_j
$$

---

### Final Result:

$$
\boxed{
\frac{\partial \mathcal{L}}{\partial z_j} = s_j - y_j
}
$$

This elegant result combines the derivative of softmax and cross-entropy into one simple expression — used in all modern deep learning frameworks.

---

### For a batch of $B$ samples:

$$
\frac{\partial \mathcal{L}}{\partial z} = \frac{1}{B}(S - Y)
$$

Where:
- $S \in \mathbb{R}^{B \times C}$: softmax outputs
- $Y \in \mathbb{R}^{B \times C}$: one-hot labels

Now, we will write methods for the layers defined above and extend the classes defined.

First, we start with the backpropagation for the Cross-Entropy + SoftMax layers.

In [ ]:
def CrossEntropyBackward(self):
  # YOUR CODE GOES HERE to compute the gradient of the cross-entropy loss with respect to the input logits
  pass

CrossEntropy.backward = CrossEntropyBackward

In [ ]:
def CrossEntropyBackward(self):
  return (self.x - self.y)/(self.x.shape[0])

CrossEntropy.backward = CrossEntropyBackward

In [ ]:
def SoftMaxBackward(self, dout):
  # dout is the gradient received by the SoftMax layer from the CrossEntropy layer - in this case, it is already the gradient that the softmax layer should return
  return dout

SoftMax.backward = SoftMaxBackward

Now, we proceed to the backpropagation function for the dense neural net layer. This receives the gradients w.r.t. the output logits, and computes the gradients w.r.t. the weights and the input.

### Derivation: Gradient of a Matrix Product

We want to compute the derivative of a matrix product:

Let:
- $A \in \mathbb{R}^{m \times n}$
- $B \in \mathbb{R}^{k \times n}$
- $Z = A B^T \in \mathbb{R}^{m \times k}$
- $\mathcal{L}$ is a scalar loss function that depends on $Z$

---

### Goal:

Compute the gradient of the loss with respect to $B$ and $A$:
$$
\frac{\partial \mathcal{L}}{\partial B}, \frac{\partial \mathcal{L}}{\partial A}
$$

---

### Step-by-Step Derivation

Let’s take a single entry from $Z$:

$$
Z_{ij} = \sum_{l=1}^{n} A_{i,l} B_{j,l}
= A_{i,:} \cdot B_{j,:}^T
$$

Then by the chain rule:

$$
\frac{\partial \mathcal{L}}{\partial B_{j,l}} = \sum_{i=1}^{m} \frac{\partial \mathcal{L}}{\partial Z_{i,j}} \cdot \frac{\partial Z_{i,j}}{\partial B_{j,l}}
$$

$$
\frac{\partial \mathcal{L}}{\partial A_{i,l}} = \sum_{j=1}^{k} \frac{\partial \mathcal{L}}{\partial Z_{i,j}} \cdot \frac{\partial Z_{i,j}}{\partial A_{i,l}}
$$

From earlier:

$$
\frac{\partial Z_{i,j}}{\partial B_{j,l}} = A_{i,l}
$$
$$
\frac{\partial Z_{i,j}}{\partial A_{i,l}} = B_{j,l}
$$

So:

$$
\frac{\partial \mathcal{L}}{\partial B_{j,l}} = \sum_{i=1}^{m} \frac{\partial \mathcal{L}}{\partial Z_{i,j}} \cdot A_{i,l}
$$

This is equivalent to a matrix multiplication between:

- Row $j$ of $(\frac{\partial \mathcal{L}}{\partial Z})^T$
- Column $l$ of $A$

$$
\frac{\partial \mathcal{L}}{\partial A_{i,l}} = \sum_{j=1}^{k} \frac{\partial \mathcal{L}}{\partial Z_{i,j}} \cdot B_{j,l}
$$

This is equivalent to a matrix multiplication between:

- Row $i$ of $\frac{\partial \mathcal{L}}{\partial Z}$
- Column l of B

---

### Final Result (Matrix Form)

Putting all entries together into a matrix:

$$
\boxed{
\frac{\partial \mathcal{L}}{\partial B} = \left( \frac{\partial \mathcal{L}}{\partial Z} \right)^T A
}
$$
####$$
\boxed{
\frac{\partial \mathcal{L}}{\partial A} = \left( \frac{\partial \mathcal{L}}{\partial Z} \right) B
}
$$
---

This identity is widely used in neural networks when computing the gradient of a linear (dense) layer:

$$
Z = X W^T
$$
$$
\quad \Rightarrow \quad \frac{\partial \mathcal{L}}{\partial W} = \left( \frac{\partial \mathcal{L}}{\partial Z} \right)^T X
$$
$$
\quad \Rightarrow \quad \frac{\partial \mathcal{L}}{\partial X} =  \left( \frac{\partial \mathcal{L}}{\partial Z} \right) W
$$

In [ ]:
def DenseNNBackward(self, dout):
    B = dout.shape[0]

    # Gradient w.r.t. weights: (Out_features x Batch_size) @ (Batch_size x (In_features+1)) → (Out_features x In_features+1)
    # YOUR CODE GOES HERE to compute the gradient of the loss with respect to the weights
    self.grad_weights = None # Replace with your code

    # Optional: Gradient clipping
    norm = np.linalg.norm(self.grad_weights)
    if norm > 1:
        self.grad_weights *= (1 / norm)

    # Gradient w.r.t. input: (Batch_size x Out_features) @ (Out_features x In_features+1) → (Batch_size x In_features+1)
    # YOUR CODE GOES HERE to compute the gradient of the loss with respect to the augmented input
    dx_aug = None # Replace with your code

    # Drop bias column
    return dx_aug[:, 1:]  # shape: (Batch_size x In_features)


DenseNNLayer.backward = DenseNNBackward

In [ ]:
def DenseNNBackward(self, dout):
    B = dout.shape[0]

    # Gradient w.r.t. weights: (Out_features x Batch_size) @ (Batch_size x (In_features+1)) → (Out_features x In_features+1)
    self.grad_weights = (dout.T @ self.x_augmented)

    # Optional: Gradient clipping
    norm = np.linalg.norm(self.grad_weights)
    if norm > 1:
        self.grad_weights *= (1 / norm)

    # Gradient w.r.t. input: (Batch_size x Out_features) @ (Out_features x In_features+1) → (Batch_size x In_features+1)
    dx_aug = dout @ self.weights  # same as dout · W

    # Drop bias column
    return dx_aug[:, 1:]  # shape: (Batch_size x In_features)


DenseNNLayer.backward = DenseNNBackward

Next we, do the backprop through the flatten layer, which is pretty elementary.

In [ ]:
def flattenBackward(self, dout):
  return dout.reshape(self.original_shape)

Flatten.backward = flattenBackward

Now, we move on to the backprop through the ReLU activation function. It is simple to compute the derivative of the output w.r.t the input of the ReLU layer, and is left as an exercise.

In [ ]:
def ReLUBackward(self, dout):
  # YOUR CODE GOES HERE to compute the gradient of the loss with respect to the input
  pass

ReLU.backward = ReLUBackward

In [ ]:
def ReLUBackward(self, dout):
  return dout * (self.x>0)

ReLU.backward = ReLUBackward

Now, we wish to backpropagate through the MaxPool layer. An intuitive way to think about the derivatives needed, is that the output of the MaxPool layer equals the input for the positions that are the maximums in their respective chunks, while the output is independent of all other inputs. Now, it should be clear why we stored the indices of the maximum values of each chunk during the forward pass.

In [ ]:
def MaxPoolBackward(self, dout):
    dx = np.zeros_like(self.max_mask)  # same shape as input x

    batch_size, channels, out_height, out_width = dout.shape

    # YOUR CODE GOES HERE to iterate through the output and distribute the gradient to the correct input location
        # YOUR CODE GOES HERE to get the stored maximum index
        # YOUR CODE GOES HERE to assign the gradient from the output to the input at the maximum index

    return dx

MaxPool.backward = MaxPoolBackward

In [ ]:
def MaxPoolBackward(self, dout):
    dx = np.zeros_like(self.max_mask)  # same shape as input x

    batch_size, channels, out_height, out_width = dout.shape

    for b in range(batch_size):
        for c in range(channels):
            for i in range(out_height):
                for j in range(out_width):
                    i_max, j_max = self.max_indices[b, c, i, j]
                    dx[b, c, i_max, j_max] = dout[b, c, i, j]

    return dx

MaxPool.backward = MaxPoolBackward

Finally, we are at the last step for the backprop - through the convolutional layer.
### [CMU Explainer](https://deeplearning.cs.cmu.edu/F21/document/recitation/Recitation5/CNN_Backprop_Recitation_5_F21.pdf)

You may want a different reference for more clarity, the document linked has a decent explanation, and we have tried to go through the entire process step-by-step below.

Let us try to walk through the process, step-by-step. We need to compute the gradients w.r.t. the input to the convolutional layer, w.r.t. the convolutional filters, and w.r.t. the bias terms. <br><br>
Let us deal with a case with a $3 \times 3$ input matrix $X$, and a filter F of dimension $ 2\times 2$ with a stride of 1. The output in this case has dimension $2 \times 2$. Let us write the terms of the output matrix $O$ explicitly. Note that the output has a bias added to each term.<br><br>
$o_{11} = f_{11}x_{11}+f_{12}x_{12}+f_{21}x_{21}+f_{22}x_{22} + b$ <br>
$o_{12} = f_{11}x_{12}+f_{12}x_{13}+f_{21}x_{22}+f_{22}x_{23} + b$ <br>
$o_{21} = f_{11}x_{21}+f_{12}x_{22}+f_{21}x_{31}+f_{22}x_{32} + b$ <br>
$o_{22} = f_{11}x_{22}+f_{12}x_{23}+f_{21}x_{32}+f_{22}x_{33} + b$ <br><br>
Now, we are provided the derivative of the loss w.r.t. to the output matrix O, i.e. $\frac{\partial \mathcal{L}}{\partial o_{11}}, \frac{\partial \mathcal{L}}{\partial o_{12}}, \frac{\partial \mathcal{L}}{\partial o_{21}}, \frac{\partial \mathcal{L}}{\partial o_{22}}$ are known. We wish to compute the derivative w.r.t. $F$ and $X$, and the bias $b$. <br><br>
Let us start with the derivative $\frac{\partial \mathcal{L}}{\partial b}$.
From the chain rule, <br><br>
$$\frac{\partial \mathcal{L}}{\partial b} = \sum_{i,j}\frac{\partial \mathcal{L}}{\partial o_{ij}}\frac{\partial o_{ij}}{\partial b}, i, j = {1, 2}$$ <br>
$$\frac{\partial o_{ij}}{\partial b} = 1\ ∀ i,j$$ <br>
**So, the derivative of the loss function w.r.t. the bias is just the sum of the derivatives w.r.t. the output matrix terms.** <br><br>

---

Now let us compute the derivative of the loss function w.r.t. the **filter weights** \( F \).  
Recall that the filter is:

$$
F =
\begin{bmatrix}
f_{11} & f_{12} \\
f_{21} & f_{22}
\end{bmatrix}
$$

We want to compute:

$$
\frac{\partial \mathcal{L}}{\partial f_{11}}, \quad
\frac{\partial \mathcal{L}}{\partial f_{12}}, \quad
\frac{\partial \mathcal{L}}{\partial f_{21}}, \quad
\frac{\partial \mathcal{L}}{\partial f_{22}}
$$

Let’s start with $ \frac{\partial \mathcal{L}}{\partial f_{11}} $.  
From the output expressions, we can see that $ f_{11} $ appears in all of these:

- $ o_{11} = f_{11}x_{11} + \dots $
- $ o_{12} = f_{11}x_{12} + \dots $
- $ o_{21} = f_{11}x_{21} + \dots $
- $ o_{22} = f_{11}x_{22} + \dots $

So by the chain rule:

$$
\frac{\partial \mathcal{L}}{\partial f_{11}} =
\frac{\partial \mathcal{L}}{\partial o_{11}} \cdot x_{11} +
\frac{\partial \mathcal{L}}{\partial o_{12}} \cdot x_{12} +
\frac{\partial \mathcal{L}}{\partial o_{21}} \cdot x_{21} +
\frac{\partial \mathcal{L}}{\partial o_{22}} \cdot x_{22}
$$

Similarly, for the other filter weights:

$$
\frac{\partial \mathcal{L}}{\partial f_{12}} =
\frac{\partial \mathcal{L}}{\partial o_{11}} \cdot x_{12} +
\frac{\partial \mathcal{L}}{\partial o_{12}} \cdot x_{13} +
\frac{\partial \mathcal{L}}{\partial o_{21}} \cdot x_{22} +
\frac{\partial \mathcal{L}}{\partial o_{22}} \cdot x_{23}
$$

$$
\frac{\partial \mathcal{L}}{\partial f_{21}} =
\frac{\partial \mathcal{L}}{\partial o_{11}} \cdot x_{21} +
\frac{\partial \mathcal{L}}{\partial o_{12}} \cdot x_{22} +
\frac{\partial \mathcal{L}}{\partial o_{21}} \cdot x_{31} +
\frac{\partial \mathcal{L}}{\partial o_{22}} \cdot x_{32}
$$

$$
\frac{\partial \mathcal{L}}{\partial f_{22}} =
\frac{\partial \mathcal{L}}{\partial o_{11}} \cdot x_{22} +
\frac{\partial \mathcal{L}}{\partial o_{12}} \cdot x_{23} +
\frac{\partial \mathcal{L}}{\partial o_{21}} \cdot x_{32} +
\frac{\partial \mathcal{L}}{\partial o_{22}} \cdot x_{33}
$$

So, the gradient of the loss with respect to each filter weight is just the sum of all the input pixels that were multiplied by it in the forward pass, each scaled by the gradient of the corresponding output.

---

Now let’s compute the derivative of the loss with respect to the **input matrix** $X$.

Each input value contributes to multiple outputs. Let’s look at one example: $x_{22}$.

It appears in:

- $o_{11}$ via $f_{22}$  
- $o_{12}$ via $f_{21}$  
- $o_{21}$ via $f_{12}$  
- $o_{22}$ via $f_{11}$  

So the gradient is:

$$
\frac{\partial \mathcal{L}}{\partial x_{22}} =
\frac{\partial \mathcal{L}}{\partial o_{11}} \cdot f_{22} +
\frac{\partial \mathcal{L}}{\partial o_{12}} \cdot f_{21} +
\frac{\partial \mathcal{L}}{\partial o_{21}} \cdot f_{12} +
\frac{\partial \mathcal{L}}{\partial o_{22}} \cdot f_{11}
$$

Each input gradient is the sum of all the gradients it "receives" from outputs that used it, weighted by the filter values that used it.

---


Let’s continue by computing the **gradient of the loss with respect to all values of the input matrix** $X$.

We know from earlier that each $x_{ij}$ contributes to multiple outputs due to the sliding window, and therefore receives gradients from multiple positions in the output matrix, scaled by corresponding filter weights.

We compute a few examples explicitly:

---

### $x_{11}$

$x_{11}$ only contributes to $o_{11}$ via $f_{11}$:

$$
\frac{\partial \mathcal{L}}{\partial x_{11}} =
\frac{\partial \mathcal{L}}{\partial o_{11}} \cdot f_{11}
$$

---

### $x_{12}$

$x_{12}$ contributes to:

- $o_{11}$ via $f_{12}$
- $o_{12}$ via $f_{11}$

So:

$$
\frac{\partial \mathcal{L}}{\partial x_{12}} =
\frac{\partial \mathcal{L}}{\partial o_{11}} \cdot f_{12} +
\frac{\partial \mathcal{L}}{\partial o_{12}} \cdot f_{11}
$$

---

### $x_{13}$

$x_{13}$ only appears in $o_{12}$ via $f_{12}$:

$$
\frac{\partial \mathcal{L}}{\partial x_{13}} =
\frac{\partial \mathcal{L}}{\partial o_{12}} \cdot f_{12}
$$

---

### $x_{21}$

$x_{21}$ contributes to:

- $o_{11}$ via $f_{21}$
- $o_{21}$ via $f_{11}$

So:

$$
\frac{\partial \mathcal{L}}{\partial x_{21}} =
\frac{\partial \mathcal{L}}{\partial o_{11}} \cdot f_{21} +
\frac{\partial \mathcal{L}}{\partial o_{21}} \cdot f_{11}
$$

---

###$x_{22}$

Previously shown:

$$
\frac{\partial \mathcal{L}}{\partial x_{22}} =
\frac{\partial \mathcal{L}}{\partial o_{11}} \cdot f_{22} +
\frac{\partial \mathcal{L}}{\partial o_{12}} \cdot f_{21} +
\frac{\partial \mathcal{L}}{\partial o_{21}} \cdot f_{12} +
\frac{\partial \mathcal{L}}{\partial o_{22}} \cdot f_{11}
$$

---

### $x_{23}$

$$
\frac{\partial \mathcal{L}}{\partial x_{23}} =
\frac{\partial \mathcal{L}}{\partial o_{12}} \cdot f_{22} +
\frac{\partial \mathcal{L}}{\partial o_{22}} \cdot f_{12}
$$

---

### $x_{31}$

$$
\frac{\partial \mathcal{L}}{\partial x_{31}} =
\frac{\partial \mathcal{L}}{\partial o_{21}} \cdot f_{21}
$$

---

### $x_{32}$

$$
\frac{\partial \mathcal{L}}{\partial x_{32}} =
\frac{\partial \mathcal{L}}{\partial o_{21}} \cdot f_{22} +
\frac{\partial \mathcal{L}}{\partial o_{22}} \cdot f_{21}
$$

---

### $x_{33}$

$$
\frac{\partial \mathcal{L}}{\partial x_{33}} =
\frac{\partial \mathcal{L}}{\partial o_{22}} \cdot f_{22}
$$

---

##Summary: Input Gradients as a Convolution

What we just did manually can be **efficiently computed** by:

- **Flipping the filter** horizontally and vertically
- Performing a **full convolution** (i.e., stride 1 and zero-padding so the output is the same size as the input)

Let:

- $O'$ be the matrix of gradients of the loss with respect to output:  
  $O' = \frac{\partial \mathcal{L}}{\partial O}$
- $F^\text{rot}$ be the 180°-rotated filter (i.e., flipped over both axes)

Then:

$$
\frac{\partial \mathcal{L}}{\partial X} = O' * F^\text{rot}
$$

Where $*$ denotes **full convolution**.

---

## Summary: Filter Gradients as a Convolution

The gradient w.r.t. the filter can also be expressed as:

- A **valid convolution** of the input $X$ with the output gradient $O'$

That is:

$$
\frac{\partial \mathcal{L}}{\partial F} = X * O'
$$

Where this is a **valid convolution** (no padding).

---

Both gradients can be computed efficiently using built-in convolution functions during backpropagation.
Let us now implement this using NumPy.

In [ ]:
def ConvBackward(self, dout):
    batch_size, out_channels, out_height, out_width = dout.shape
    _, in_channels, H_padded, W_padded = self.x.shape
    k = self.kernel_size
    stride = self.stride
    pad = self.padding

    # Bias gradient
    # YOUR CODE GOES HERE to compute the bias gradient
    self.grad_bias = None # Replace with your code
    norm_b = np.linalg.norm(self.grad_bias)
    if norm_b > 1:
        self.grad_bias *= (1 / norm_b) #normalization to stabilize values while preserving direction

    # Weight gradients
    # YOUR CODE GOES HERE to compute the weight gradients
    self.grad_weights = np.zeros_like(self.weights) # Replace with your code
    norm_w = np.linalg.norm(self.grad_weights)
    if norm_w > 1:
        self.grad_weights *= (1 / norm_w)

    # Gradient w.r.t. input
    # YOUR CODE GOES HERE to compute the input gradient
    dx = np.zeros_like(self.x) # Replace with your code

    # Return only the center (unpadded) region that corresponds to the original unpadded input x
    if pad > 0:
        return dx[:, :, pad:-pad, pad:-pad]
    else:
        return dx

Conv.backward = ConvBackward

In [ ]:
def ConvBackward(self, dout):
    batch_size, out_channels, out_height, out_width = dout.shape
    _, in_channels, H_padded, W_padded = self.x.shape
    k = self.kernel_size
    stride = self.stride
    pad = self.padding

    # Bias gradient
    self.grad_bias = np.sum(dout, axis=(0, 2, 3)) #we add for the whole batch, and this can be treated as an average because when initializing dout, we had already divided by batch_size
    norm_b = np.linalg.norm(self.grad_bias)
    if norm_b > 1:
        self.grad_bias *= (1 / norm_b) #normalization to stabilize values while preserving direction

    # Weight gradients
    # We implement a valid convolution as described above
    self.grad_weights = np.zeros_like(self.weights)
    for b in range(batch_size):
        x_b = self.x[b]
        for i in range(out_height):
            for j in range(out_width):
                i_start = i * stride
                j_start = j * stride
                patch = x_b[:, i_start:i_start + k, j_start:j_start + k]
                for c in range(out_channels):
                    self.grad_weights[c] += patch * dout[b, c, i, j]

    norm_w = np.linalg.norm(self.grad_weights)
    if norm_w > 1:
        self.grad_weights *= (1 / norm_w)

    # Gradient w.r.t. input
    dx = np.zeros_like(self.x)

    # Pad dout with (k - 1) for full conv (k is kernel size)
    pad_amt = k - 1
    dout_padded = np.pad(dout, ((0, 0), (0, 0), (pad_amt, pad_amt), (pad_amt, pad_amt)), mode='constant')

    for b in range(batch_size):
        for c_in in range(in_channels):
            for c_out in range(out_channels):
                flipped_filter = np.flip(self.weights[c_out, c_in], axis=(0, 1))  # (k, k)
                for i in range(H_padded):
                    for j in range(W_padded):
                        region = dout_padded[b, c_out, i:i + k, j:j + k]
                        dx[b, c_in, i, j] += np.sum(region * flipped_filter)

    # Return only the center (unpadded) region that corresponds to the original unpadded input x
    if pad > 0:
        return dx[:, :, pad:-pad, pad:-pad]
    else:
        return dx

Conv.backward = ConvBackward

Great, we are now ready with a full forward and backward pass through our network. 2 layers - convolutional layer and the dense NN layer have parameters that require updating using the calculated gradients. Let us quickly implement the updation, before finally setting up our model that is based on the original LeNet-5 architecture, and training it on the MNIST dataset.

In [ ]:
def Conv_apply_gradients(self, lr):
    self.weights -= lr * self.grad_weights
    self.bias -= lr * self.grad_bias
Conv.apply_gradients = Conv_apply_gradients

def NN_apply_gradients(self, lr):
  self.weights -= lr * self.grad_weights
DenseNNLayer.apply_gradients = NN_apply_gradients

LeNet-5 Architecture Explained

LeNet-5 is a pioneering convolutional neural network (CNN) architecture developed by Yann LeCun in 1998 for handwritten digit recognition (e.g., MNIST). It consists of 7 layers (excluding input), all with learnable parameters. The network follows a structured pipeline:
1.	Input Layer: Takes a $32 \times 32$ grayscale image. (MNIST images are $28 \times 28$, so they’re typically padded with zeros to match this size.)
2.	C1 - Convolutional Layer: Applies 6 filters of size $5 \times 5$, resulting in 6 feature maps of size $28 \times 28$. (No padding used, stride = 1.)
3.	S2 - Subsampling Layer: Performs average pooling over $2 \times 2$ regions with stride 2, reducing each feature map to size $14 \times 14$.
4.	C3 - Convolutional Layer: Applies 16 filters of size $5 \times 5$. Not all filters are connected to all previous feature maps — this forms a sparse connection pattern. Output size: $10 \times 10$ feature maps.
5.	S4 - Subsampling Layer: Again does average pooling, reducing to $5 \times 5$ feature maps.
6.	C5 - Convolutional Layer: Applies 120 filters of size $5 \times 5$, covering the entire $5 \times 5$ input — each output is a single value → yielding a 120-dimensional vector.
7.	F6 - Fully Connected Layer: A dense layer with 84 units, followed by a final output layer with 10 neurons (for digit classes 0–9).

LeNet-5 combines convolutional feature extraction with simple pooling and fully connected classification, laying the foundation for modern deep learning architectures.

Here, we shall implement a similar structure, using maxpooling instead of average pooling.

First, we load and pre-process the data.

In [ ]:
from tensorflow.keras.datasets import mnist

# Load data
(train_X, train_y), (test_X, test_y) = mnist.load_data()

# Normalize input to [0,1]
train_X = train_X.astype(np.float32) / 255.0
test_X = test_X.astype(np.float32) / 255.0

# Add channel dimension
train_X = train_X[:, None, :, :]   # shape: (B, 1, 28, 28)
test_X = test_X[:, None, :, :]

# Pad to (1, 32, 32) to match LeNet
train_X = np.pad(train_X, ((0, 0), (0, 0), (2, 2), (2, 2)))
test_X = np.pad(test_X, ((0, 0), (0, 0), (2, 2), (2, 2)))

# One-hot encode labels
train_y_oh = np.eye(10)[train_y]   # shape: (B, 10)
test_y_oh = np.eye(10)[test_y]

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Now, we write a function to create batches.

In [ ]:
def create_batches(X, y, batch_size):
    for i in range(0, len(X), batch_size):
        yield X[i:i+batch_size], y[i:i+batch_size]

It's time to define the model, based on LeNet architecture

In [ ]:
model = [
    Conv(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0),
    MaxPool(kernel_size=2, stride=2),
    Conv(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
    MaxPool(kernel_size=2, stride=2),
    Flatten(),
    DenseNNLayer(input_features=400, output_features=120),
    ReLU(),
    DenseNNLayer(input_features=120, output_features=84),
    ReLU(),
    DenseNNLayer(input_features=84, output_features=10),
    SoftMax()
]

Now, we write the training loop by creating batches, then performing forward and backward passes through the defined model.

In [ ]:
from tqdm import tqdm
num_epochs = 5
loss_fn = CrossEntropy()
learning_rate = 0.01
batch_size = 16
num_samples = len(train_X)

for epoch in range(num_epochs):
    total_loss = 0
    correct = 0
    total = 0

    # Wrap batches with tqdm for progress bar
    batch_iterator = tqdm(create_batches(train_X, train_y_oh, batch_size), total=num_samples//batch_size, desc=f"Epoch {epoch+1}/{num_epochs}")

    for x_batch, y_batch in batch_iterator:
        # YOUR CODE GOES HERE for the forward pass
        out = x_batch
        for layer in model:
            pass

        # YOUR CODE GOES HERE to calculate the loss

        # YOUR CODE GOES HERE for the backward pass
        dout = loss_fn.backward()
        for layer in reversed(model):
            pass
        # YOUR CODE GOES HERE to apply gradients
        for layer in model:
            pass


        # Update progress bar info
        batch_iterator.set_postfix(loss=loss, acc=f"{correct/total:.4f}")

    avg_loss = total_loss / total
    accuracy = correct / total
    print(f"Epoch {epoch+1} completed - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

In [ ]:
from tqdm import tqdm
num_epochs = 5
loss_fn = CrossEntropy()
learning_rate = 0.01
batch_size = 16
num_samples = len(train_X)

for epoch in range(num_epochs):
    total_loss = 0
    correct = 0
    total = 0

    # Wrap batches with tqdm for progress bar
    batch_iterator = tqdm(create_batches(train_X, train_y_oh, batch_size), total=num_samples//batch_size, desc=f"Epoch {epoch+1}/{num_epochs}")

    for x_batch, y_batch in batch_iterator:
        out = x_batch
        for layer in model:
            out = layer.forward(out)

        loss = loss_fn.forward(out, y_batch)
        total_loss += loss * len(x_batch)

        preds = np.argmax(out, axis=1)
        targets = np.argmax(y_batch, axis=1)
        correct += np.sum(preds == targets)
        total += len(x_batch)

        dout = loss_fn.backward()
        for layer in reversed(model):
            dout = layer.backward(dout)

        for layer in model:
            if hasattr(layer, "apply_gradients"):
                layer.apply_gradients(learning_rate)

        # Update progress bar info
        batch_iterator.set_postfix(loss=loss, acc=f"{correct/total:.4f}")

    avg_loss = total_loss / total
    accuracy = correct / total
    print(f"Epoch {epoch+1} completed - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

Now, we write a testing function, to test our trained model.

In [ ]:
def evaluate(model, X, y, batch_size=64):
    correct = 0
    total = 0

    def create_batches(X, y, batch_size):
        for i in range(0, len(X), batch_size):
            yield X[i:i+batch_size], y[i:i+batch_size]

    for x_batch, y_batch in create_batches(X, y, batch_size):
        out = x_batch
        for layer in model:
            out = layer.forward(out)

        preds = np.argmax(out, axis=1)
        targets = np.argmax(y_batch, axis=1)

        correct += np.sum(preds == targets)
        total += len(x_batch)

    accuracy = correct / total
    print(f" Test Accuracy: {accuracy:.4f}")
    return accuracy

accuracy = evaluate(model, test_X, test_y_oh, batch_size=64)

Aaaand we're all done! Of course, we want to store the model parameters to be able to load it later.

In [ ]:
def save_model(model, filename="lenet_weights.npz"):
    params = {}
    for idx, layer in enumerate(model):
        if hasattr(layer, "weights"):
            params[f"layer_{idx}_weights"] = layer.weights
        if hasattr(layer, "bias"):
            params[f"layer_{idx}_bias"] = layer.bias
    np.savez(filename, **params)
    print(f"✅ Model parameters saved to {filename}")

In [ ]:
def load_model(model, filename="lenet_weights.npz"):
    data = np.load(filename)
    for idx, layer in enumerate(model):
        if hasattr(layer, "weights"):
            layer.weights = data[f"layer_{idx}_weights"]
        if hasattr(layer, "bias"):
            layer.bias = data[f"layer_{idx}_bias"]
    print(f"✅ Model parameters loaded from {filename}")

In [ ]:
save_model(model, "lenet_weights.npz")

In [ ]:
#optional - download the saved parameters as Colab clears its session data
from google.colab import files
files.download("lenet_weights.npz")

Update - this model, while working perfectly well, is very slow to train. The slowness is due to the nested for loops used in the forward and back pass of the convolutional and maxpooling layers. The following is an excerpt from a conversation with ChatGPT:

In the original implementation, convolution and pooling layers used nested Python loops to manually compute every patch-wise operation — iterating over the batch, channels, height, and width. This approach is highly inefficient in Python because each loop runs in pure Python (interpreted and slow), and repeated memory access (especially slicing) is costly. By contrast, vectorization leverages NumPy’s ability to perform large-scale matrix operations in compiled C under the hood, using SIMD (Single Instruction, Multiple Data) instructions that run on highly optimized BLAS libraries. For example, instead of looping over every patch to compute dot products between a filter and input, we use the im2col trick to reshape all patches into a matrix, flatten the filters, and compute all outputs at once using fast matrix multiplication (@). Similarly, for pooling, np.stride_tricks.as_strided enables efficient patch extraction in-place without copying memory, followed by np.max over axes to pool all patches simultaneously. Vectorization thus replaces thousands of slow iterations with a single batched operation, dramatically improving performance, especially for large datasets and deep models.


Let us try to understand how one particular conversion of matrix to vector is taking place in the im2col function defined below. <br>
First the input is padded as required to match the dimensions, and then the dimensions of the output are computed as out_h and out_w. <br>
The cols array is used to extract and store all the patches from the original matrix (these patches have dimensions (K, K)), and the double for loop executes this task. This double loop extracts each shifted version of the input.
For further explanation: <br>
- i, j: position in the kernel
-	For each kernel position (i, j), we take all KxK-sized patches that will align with that kernel position across all spatial locations.
- Uses strided slicing to extract efficiently:
- i:i+stride*out_h:stride picks every patch row
-	j:j+stride*out_w:stride picks every patch column
Now, we
1.	Reorder axes so each patch is first:
	-	New shape: (B, out_h, out_w, C, K, K)
2.	Flatten each patch into a row:
	-	Final shape: (B * out_h * out_w, C * K * K) — this is our im2col output.

Similar ideas are used to construct the col2im function.

In [ ]:
import numpy as np

def im2col(x, kernel_size, stride, padding):
    # Pad input: (B, C, H, W)
    x_padded = np.pad(x, ((0,0), (0,0), (padding, padding), (padding, padding)))
    B, C, H, W = x_padded.shape
    K = kernel_size
    out_h = (H - K) // stride + 1
    out_w = (W - K) // stride + 1

    cols = np.zeros((B, C, K, K, out_h, out_w))

    for i in range(K):
        for j in range(K):
            cols[:, :, i, j, :, :] = x_padded[:, :, i:i+stride*out_h:stride, j:j+stride*out_w:stride]

    cols = cols.transpose(0, 4, 5, 1, 2, 3).reshape(B*out_h*out_w, -1)
    return cols

def col2im(cols, x_shape, kernel_size, stride, padding):
    B, C, H, W = x_shape
    H_padded, W_padded = H + 2*padding, W + 2*padding
    x_padded = np.zeros((B, C, H_padded, W_padded))
    out_h = (H_padded - kernel_size) // stride + 1
    out_w = (W_padded - kernel_size) // stride + 1

    cols_reshaped = cols.reshape(B, out_h, out_w, C, kernel_size, kernel_size).transpose(0, 3, 4, 5, 1, 2)

    for i in range(kernel_size):
        for j in range(kernel_size):
            x_padded[:, :, i:i+stride*out_h:stride, j:j+stride*out_w:stride] += cols_reshaped[:, :, i, j, :, :]

    if padding > 0:
        return x_padded[:, :, padding:-padding, padding:-padding]
    else:
        return x_padded


In [ ]:
class Conv:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=1):
        self.stride = stride
        self.padding = padding
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        self.weights = np.random.randn(out_channels, in_channels, kernel_size, kernel_size) * 0.001
        self.bias = np.zeros(out_channels)

    def forward(self, x):
        self.x = x
        self.x_col = im2col(x, self.kernel_size, self.stride, self.padding)
        self.w_col = self.weights.reshape(self.out_channels, -1)
        B, C, H, W = x.shape
        out_h = (H + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_w = (W + 2 * self.padding - self.kernel_size) // self.stride + 1
        out = self.x_col @ self.w_col.T + self.bias
        out = out.reshape(B, out_h, out_w, self.out_channels).transpose(0, 3, 1, 2)
        return out

def ConvBackward(self, dout):
    B, C_out, H_out, W_out = dout.shape
    dout_reshaped = dout.transpose(0, 2, 3, 1).reshape(-1, C_out)

    # Bias gradient
    self.grad_bias = np.sum(dout_reshaped, axis=0)
    norm_b = np.linalg.norm(self.grad_bias)
    if norm_b > 1:
        self.grad_bias *= (1 / norm_b)

    # Weight gradient
    self.grad_weights = dout_reshaped.T @ self.x_col
    self.grad_weights = self.grad_weights.reshape(self.out_channels, -1, self.kernel_size, self.kernel_size)
    norm_w = np.linalg.norm(self.grad_weights)
    if norm_w > 1:
        self.grad_weights *= (1 / norm_w)

    # Input gradient
    w_flat = self.w_col
    dx_col = dout_reshaped @ w_flat
    dx = col2im(dx_col, self.x.shape, self.kernel_size, self.stride, self.padding)
    return dx

Conv.backward = ConvBackward

Similarly, we update the MaxPool layer to avoid nested for loops, and speed up processing. This time, we use a NumPy function that helps us avoid the issues we faced earlier.

In [ ]:
import numpy as np

class MaxPool:
    def __init__(self, kernel_size=2, stride=2):
        self.kernel_size = kernel_size
        self.stride = stride

    def forward(self, x):
        self.x_shape = x.shape
        B, C, H, W = x.shape
        K = self.kernel_size
        S = self.stride

        H_out = (H - K) // S + 1
        W_out = (W - K) // S + 1

        # Extract patches
        patches = np.lib.stride_tricks.as_strided(
            x,
            shape=(B, C, H_out, W_out, K, K),
            strides=(
                x.strides[0],
                x.strides[1],
                x.strides[2] * S,
                x.strides[3] * S,
                x.strides[2],
                x.strides[3],
            ),
            writeable=False
        )  # (B, C, H_out, W_out, K, K)

        # Store mask for backward
        self.patches = patches
        self.max_mask = (patches == np.max(patches, axis=(4,5), keepdims=True))

        # Compute output
        output = np.max(patches, axis=(4,5))
        return output

In [ ]:
def MaxPoolBackward(self, dout):
    B, C, H_out, W_out = dout.shape
    K = self.kernel_size
    S = self.stride

    # Initialize dx
    dx = np.zeros(self.x_shape, dtype=dout.dtype)

    # Expand dout to broadcast over (K, K)
    dout_expanded = dout[:, :, :, :, None, None]  # shape: (B, C, H_out, W_out, 1, 1)

    # Distribute dout to the max locations only
    dx_patches = self.max_mask * dout_expanded  # same shape as (B, C, H_out, W_out, K, K)

    # Now scatter dx_patches back to dx
    for i in range(K):
        for j in range(K):
            dx[:, :, i::S, j::S] += dx_patches[:, :, :, :, i, j]

    return dx
MaxPool.backward = MaxPoolBackward

You can try running the code for yourself with and without this fix - the performance gap will be very clear. During testing, the initial version took over 6 hours to train, while the second version trained on 60000 training samples and for 5 epochs in 20 minutes.